# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121436e+02     1.799224e+00
 * time: 0.09094595909118652
     1     1.088732e+01     1.107797e+00
 * time: 0.9702877998352051
     2    -1.083184e+01     1.302759e+00
 * time: 1.1034820079803467
     3    -3.359940e+01     9.121144e-01
 * time: 1.2808609008789062
     4    -4.701631e+01     6.438061e-01
 * time: 1.439032793045044
     5    -5.664807e+01     2.353629e-01
 * time: 1.5827178955078125
     6    -5.962394e+01     1.265887e-01
 * time: 1.702692985534668
     7    -6.073524e+01     5.763839e-02
 * time: 1.8144688606262207
     8    -6.115523e+01     7.663770e-02
 * time: 1.9337899684906006
     9    -6.147165e+01     3.871426e-02
 * time: 2.0461978912353516
    10    -6.172563e+01     3.628530e-02
 * time: 2.167949914932251
    11    -6.193491e+01     3.089648e-02
 * time: 2.2822659015655518
    12    -6.201946e+01     2.700777e-02
 * time: 2.402535915374756
    13    -6.210534e+01     1.480135e-02
 * time: 2.5144259929656

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557670
    AtomicNonlocal      14.8522641
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336819

    total               -62.261666459575
